# regression ultimate code

In [1]:
import numpy as np
import argparse

import os
import math
from math import sin

import json

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
matplotlib.rc( 'savefig', facecolor = 'white' )
from matplotlib import pyplot
%matplotlib inline

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets, models
from torchsummary import summary
torch.manual_seed(0)

i_seed = 0

import sys
sys.path.append('../') # add this line so Data and data are visible in this file
sys.path.append('../../') # add this line so Data and data are visible in this file
sys.path.append('../PatchWand/') # add this line so Data and data are visible in this file

# from PatchWand import *
from plotting_tools import *
from setting import *
# from models import *
# from models_CNN import *
from evaluate import *

from stage3_preprocess import *
# from training_util import *
# from dataset_util import *
from dataIO import *
from stage4_regression import *

from importlib import reload
%load_ext autoreload
%autoreload 2

In [2]:
parser = argparse.ArgumentParser(description='SpO2_estimate')
parser.add_argument('--input_folder', metavar='input_folder', help='input_folder',
                    default='../')
parser.add_argument('--output_folder', metavar='output_folder', help='output_folder',
                    default='../')
parser.add_argument('--training_params_file', metavar='training_params_file', help='training_params_file',
                    default='training_params_list.json')


# checklist 3: comment first line, uncomment second line
args = parser.parse_args(['--input_folder', '../../data/stage4/', 
                          '--output_folder', '../../data/stage5_VO2/',
#                           '--training_params_file', 'training_params_ML.json',
#                           '--training_params_file', 'training_params_baseline.json',
                         ])
# args = parser.parse_args()
print(args)

Namespace(input_folder='../../data/stage4/', output_folder='../../data/stage5_VO2/', training_params_file='training_params_list.json')


In [3]:
inputdir = args.input_folder
outputdir = args.output_folder

if not os.path.exists(outputdir):
    os.makedirs(outputdir)

    

# get training params and dataloaders

In [4]:
# i_rep = 0
# aux_weight = 0.1
repN = 5
aux_weight = 0.3
# aux_weights = [0.001, 0.4, 0.45, 0.495, 0.4995]
# aux_weights = [0.001, 0.01, 0.1, 0.4995]
target_name = 'VO2_cosmedweighted'
aux_tasks = ['HR_patch', 'RR_cosmed']
# , 'HR_patch+RR_cosmed']

In [5]:
path_str = inputdir + 'DFR_DL1111/exp_tasks/rep{}/auxweight{}-fixed-4feat-{}-cardioresp_multiverse-CardioRespRegression-ch=1|spec=1-4dem/numeric_results/df_performance_val_{}.csv'


In [6]:
# df

In [7]:
df_performance_merged = pd.DataFrame()

for i_rep in range(repN):
    # for aux_weight in aux_weights:
    for aux_task in aux_tasks:
        
        print(aux_task, i_rep)


        df_dir = path_str.format(i_rep, aux_weight, aux_task, target_name)

        df = pd.read_csv(df_dir)

        if df.shape[0]!=17:
            print('\t', df.shape)
        df['i_rep'] = i_rep
        df['aux_task'] = aux_task

        df_performance_merged = df_performance_merged.append(df)

HR_patch 0
RR_cosmed 0
HR_patch 1
RR_cosmed 1
HR_patch 2
RR_cosmed 2
HR_patch 3
RR_cosmed 3
HR_patch 4
RR_cosmed 4


In [8]:
path_str = inputdir + 'DFR_DL1111/exp_weights/rep{}/auxweight0-fixed-4feat-HR_patch+RR_cosmed-cardioresp_multiverse-CardioRespRegression-ch=1|spec=1-4dem/numeric_results/df_performance_val_{}.csv'
aux_task = 'STL'

for i_rep in range(repN):
    # for aux_weight in aux_weights:
# for aux_task in aux_tasks:

    print(i_rep)


    # df_dir = path_str.format(i_rep, aux_weight, aux_task, target_name)
    df_dir = path_str.format(i_rep, target_name)

    df = pd.read_csv(df_dir)

    if df.shape[0]!=17:
        print('\t', df.shape)
    df['i_rep'] = i_rep
    df['aux_task'] = aux_task

    df_performance_merged = df_performance_merged.append(df)

0
1
2
3
4


In [9]:
path_str = inputdir + 'DFR_DL1111/exp_weights/rep{}/auxweight0.3-fixed-4feat-HR_patch+RR_cosmed-cardioresp_multiverse-CardioRespRegression-ch=1|spec=1-4dem/numeric_results/df_performance_val_{}.csv'
aux_task = 'HR_patch+RR_cosmed'

for i_rep in range(repN):

    print(i_rep)

    df_dir = path_str.format(i_rep, target_name)

    df = pd.read_csv(df_dir)

    if df.shape[0]!=17:
        print('\t', df.shape)
    df['i_rep'] = i_rep
    df['aux_task'] = aux_task

    df_performance_merged = df_performance_merged.append(df)

0
1
2
3
4


In [10]:
df_performance_merged

,CV,task,Rsquared,PCC,rmse,mae,mape,i_rep,aux_task
0,105,VO2_cosmedweighted,0.071092,0.613950,2.290849,1.794352,0.233051,0,HR_patch
1,115,VO2_cosmedweighted,0.709623,0.887061,2.330113,1.947111,0.574955,0,HR_patch
2,107,VO2_cosmedweighted,-0.225565,0.618323,3.894165,3.061416,0.342667,0,HR_patch
3,117,VO2_cosmedweighted,0.412176,0.844639,2.836372,2.416046,0.344345,0,HR_patch
4,106,VO2_cosmedweighted,0.759062,0.923930,2.038420,1.472982,0.194462,0,HR_patch
...,...,...,...,...,...,...,...,...,...
12,111,VO2_cosmedweighted,0.366065,0.896454,2.111966,1.622658,0.204855,4,HR_patch+RR_cosmed
13,118,VO2_cosmedweighted,0.849035,0.922346,1.834405,1.423136,0.227044,4,HR_patch+RR_cosmed
14,120,VO2_cosmedweighted,0.595905,0.824527,2.267525,1.648608,0.338391,4,HR_patch+RR_cosmed
15,121,VO2_cosmedweighted,0.740308,0.930325,2.136204,1.819393,0.301105,4,HR_patch+RR_cosmed


In [11]:
metric_name = 'PCC'
metric_name = 'PCC'
metric_name = 'rmse'

df_table = df_performance_merged.groupby(['aux_task', 'i_rep']).mean()[[metric_name]]

In [12]:
df_table

rmse
aux_task           i_rep          
HR_patch           0      3.037847
                   1      3.124491
                   2      3.317377
                   3      3.367221
                   4      3.166272
HR_patch+RR_cosmed 0      2.703623
                   1      2.644451
                   2      3.040849
                   3      2.705091
                   4      2.802994
RR_cosmed          0      3.225208
                   1      3.289272
                   2      3.190572
                   3      3.228725
                   4      2.857328
STL                0      3.658434
                   1      4.587235
                   2      3.208151
                   3      4.281606
                   4      3.029889

In [13]:
df_table_mean = df_table.reset_index(['aux_task', 'i_rep']).groupby(['aux_task']).mean()[[metric_name]]
df_table_std = df_table.reset_index(['aux_task', 'i_rep']).groupby(['aux_task']).std()[[metric_name]]

df_table_mean =  df_table_mean.reset_index(['aux_task'])
df_table_std =  df_table_std.reset_index(['aux_task'])

display(df_table_mean)
display(df_table_std)


,aux_task,rmse
0,HR_patch,3.202642
1,HR_patch+RR_cosmed,2.779402
2,RR_cosmed,3.158221
3,STL,3.753063


,aux_task,rmse
0,HR_patch,0.136783
1,HR_patch+RR_cosmed,0.156834
2,RR_cosmed,0.171912
3,STL,0.671577


In [14]:
df_table_merged = pd.DataFrame()

for aux_task in df_table_mean['aux_task'].unique():
    # print(aux_task)
    
    merged_mean = df_table_mean[df_table_mean['aux_task']==aux_task][metric_name].values[0]
    merged_std = df_table_std[df_table_mean['aux_task']==aux_task][metric_name].values[0]
    
    performance_dict = {
        'aux_task': aux_task,
        'merged_stats': '{:.2f}±{:.2f}'.format(merged_mean, merged_std),
    }
    
    df_table_merged = df_table_merged.append(pd.DataFrame(performance_dict, index=[0]))
    
df_table_merged.set_index(['aux_task'])
# .loc[aux_tasks]


,merged_stats
aux_task,
HR_patch,3.20±0.14
HR_patch+RR_cosmed,2.78±0.16
RR_cosmed,3.16±0.17
STL,3.75±0.67
